# Welcom to EdgeLab for Google Colab Training Example 🔥 

<a href="https://colab.research.google.com/github/Seeed-Studio/EdgeLab/blob/main/notebooks/Google-Colab-YOLOv5-A1101-Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> **[🚀🚀🚀 One-Click to Deploy to Google Colab 🚀🚀🚀](https://colab.research.google.com/github/Seeed-Studio/EdgeLab/blob/main/notebooks/Google-Colab-YOLOv5-A1101-Example.ipynb)**

In this tutorial, we will demonstrate how to use [EdgeLab](https://github.com/Seeed-Studio/EdgeLab) and Colab to train a YOLOv5-Tiny model for meter reading ⏱️.

Table of contents:

- [Setup EdgeLab](#setup-edgelab)
- [Download Custom Meter Datasets](#download-custom-meter-datasets)
- [Train YOLOv5-Tiny Model](#train-yolov5-tiny-model)
- [Optimize YOLOv5-Tiny Model and Export](#optimize-yolov5-tiny-model-and-export)
- [Deploy to SenseCAP A1101](#deploy-to-sensecap-a1101)
- [Thanks for Trying out EdgeLab 🎉](#thanks-for-trying-out-edgelab-🎉)

For more details of YOLOv5 model or application scenarios, please refer to [EdgeLab Documentation](https://seeed-studio.github.io/EdgeLab/tutorials/training/yolov5).

**Tips: Since model training consumes a lot of computational resources, we recommend that you run this notebook on [Colab's GPU instances](https://research.google.com/colaboratory/faq.html#gpu-availability).**

## Setup EdgeLab

**Step 0:** Clone the EdgeLab source code from [EdgeLab GitHub repository](https://github.com/Seeed-Studio/EdgeLab), and enter the EdgeLab project directory.

In [ ]:
!git clone https://github.com/Seeed-Studio/EdgeLab.git  # currently we're using experimental 2.0 version branch

In [ ]:
%cd /content/EdgeLab

**Step 1:** Use `python3.8` as default python in colab, then install corresponding version python and pip package.

In [ ]:
%env PYTHON_EXEC=python3.8

In [ ]:
!sudo apt-get update
!sudo apt-get install ${PYTHON_EXEC}-dev python3-pip -y

**Step 2:** Use `scripts/setup_colab.sh` to automatically setup EdgeLab dependencies.

In [ ]:
!chmod +x scripts/setup_colab.sh
!scripts/setup_colab.sh $(which ${PYTHON_EXEC})

## Download Custom Meter Datasets

We'll download our dataset from Roboflow. Use the "**COCO**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).
<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

- open our digital meter dataset [seeed_meter_digit](https://universe.roboflow.com/seeeddatasets/seeed_meter_digit/)
- Download Dataset with YOLOv5 format
- Copy the download code and paste it into the below

***More Seeed Studio public datasets are available [here](https://universe.roboflow.com/seeeddatasets).***

In [ ]:
!mkdir -p datasets
!wget "https://universe.roboflow.com/ds/GDH5ex194I?key=dPovYB5xI6" -O datasets/digital_meter.zip
!unzip -q datasets/digital_meter.zip -d datasets/digital_meter && rm datasets/digital_meter.zip

## Train YOLOv5 Tiny Model

### Examine the YOLOv5 Tiny config file

The config we use for train is located at `configs/yolov5/yolov5_tiny_1xb16_300e_coco.py`.

For more config details, please refer to [EdgeLab Documentation - Config](https://seeed-studio.github.io/EdgeLab/tutorials/config).

In [ ]:
!cat configs/yolov5/yolov5_tiny_1xb16_300e_coco.py

### Train the model using default config

Then, train YOLOv5-Tiny model using default config, note here we need to override the config by 2 parameters:

- `data_root` - the datasets path, which located at path `datasets/digital_meter`

- `max_epochs`- the train epochs, we use `50` to reduce the training time

- `num_classes` - the calsses number of datasets, we use `11` here ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'N']

For more training details, please refer to [EdgeLab Documentation - Train YOLOv5-Tiny Model](https://seeed-studio.github.io/EdgeLab/tutorials/training/yolov5).

In [ ]:
!${PYTHON_EXEC} tools/train.py \
  det \
  configs/yolov5/yolov5_tiny_1xb16_300e_coco.py \
  --cfg-options \
    max_epochs=50 \
    num_classes=11 \
    data_root='datasets/digital_meter/'

The output directory in the training process would be look like this:

In [ ]:
!ls -alh work_dirs/yolov5_tiny_1xb16_300e_coco

We could inspect the latest model weight path by this command:

In [ ]:
!echo "Latest model weights path: $(cat work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint)"

### Validate the Model

We recommend you to validate the model with the latest weight after training using the `tool/test.py` script.

In [ ]:
!${PYTHON_EXEC} tools/test.py \
    det \
    configs/yolov5/yolov5_tiny_1xb16_300e_coco.py \
    "$(cat work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint)" \
    --dump work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint.pkl \
    --cfg-options \
        data_root='datasets/digital_meter'

In [ ]:
import pickle

with open('work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint.pkl', 'rb') as f:
    data = pickle.load(f)
    for k in data.keys():
      print(f'{k}: {data[k]:.5f}')

## Optimize YOLOv5-Tiny Model and Export

### Export YOLOv5-Tiny PyTorch model to TFLite model

Here we're using `tools/torch2tflite.py` to convert and export the latest weights to a TFLite model at a INT8 precision for inference.

In [ ]:
!${PYTHON_EXEC} tools/torch2tflite.py \
  configs/yolov5/yolov5_tiny_1xb16_300e_coco.py \
  --checkpoint $(cat work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint) \
  --type int8 \
  --cfg-options \
    data_root='datasets/digital_meter/'

## Deploy to SenseCAP A1101

This example is a tutorial for deploying the models from [EdgeLab](https://github.com/Seeed-Studio/Edgelab/) to Grove - Vision AI module, based on the [Synopsys GUN Toolchain](https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain) and [Tensorflow Lite Micro](https://github.com/tensorflow/tflite-micro) implementations.

For more details, please refer to [EdgeLab Documentation - Example - Grove - Deploy](https://seeed-studio.github.io/EdgeLab/examples/grove/deploy#compile-and-deploy).

### Download and Setup Build Tool Chain

In [ ]:
!wget https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain/releases/download/arc-2020.09-release/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz -P ~/

In [ ]:
!tar -zxf ~/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz --directory ~/

In [ ]:
SYSTEM_PATH = %env PATH
SYSTEM_PATH = SYSTEM_PATH.replace('"', '')
SYSTEM_HOME = %env HOME
SYSTEM_HOME = SYSTEM_HOME.replace('"', '')
SYSTEM_PATH = f'{SYSTEM_HOME}/arc_gnu_2020.09_prebuilt_elf32_le_linux_install/bin:{SYSTEM_PATH}'
%set_env PATH={SYSTEM_PATH}

### Clone and Setup SenseCAP A1101 SDK

In [ ]:
!git clone https://github.com/Seeed-Studio/edgelab-example-vision-ai example/grove

In [ ]:
!${PYTHON_EXEC} -m pip install numpy requests colorama serial pyserial

### Build Firmware 

In [ ]:
!cd example/grove && \
  make HW=sensecap_vision_ai APP=digital_meter && make flash

### Convert Firmware Image to UF2

In [ ]:
!cd example/grove && \
  ${PYTHON_EXEC} tools/ufconv/uf2conv.py \
    -t 0 \
    -c tools/image_gen_cstm/output/output.img \
    -o firmware.uf2

In [ ]:
!echo "The UF2 firmware is located at: $(pwd)/example/grove/firmware.uf2"

In [ ]:
from IPython.display import FileLink

FileLink('example/grove/firmware.uf2')

### Convert Trained Model (TFLite) to UF2

In [ ]:
!cd example/grove && \
  ${PYTHON_EXEC} tools/ufconv/uf2conv.py \
    -t 18 \
    -c "$(cat ../../work_dirs/yolov5_tiny_1xb16_300e_coco/last_checkpoint)_int8.tflite" \
    -o model.uf2

In [ ]:
!echo "The UF2 model is located at: $(pwd)/example/grove/model.uf2"

In [ ]:
FileLink('example/grove/model.uf2')

Please download the UF2 Firmware and UF2 Model and flash them into a SenseCAP A1101 module. Finally, you can see the real-time meter reading results as shown in the figure below.

![Meter Reader](https://seeed-studio.github.io/EdgeLab/static/grove/images/digital_meter.gif)

For more information on **how to use these UF2 images**, please refer to [EdgeLab Documentations - Deploy - Grove - Deployment Routines](https://seeed-studio.github.io/EdgeLab/examples/grove/deploy#deployment-routines).

## Thanks for Trying Out EdgeLab 🎉

Congratulations, you have completed this tutorial. If you are interested in more application scenarios or our projects, please feel free to give [EdgeLab](https://github.com/Seeed-Studio/EdgeLab) a star ✨ on GitHub.

If you have any questions about this tutorial, please also feel free to [submit an issue](https://github.com/Seeed-Studio/EdgeLab/issues).